In [1]:
# import SharedArray as sa
# ROBOT_TYPE_NAME = 'indy7'
# sa.delete("shm://{}.prepared".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.grasp_img".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.arm_img".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.rh_vals".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.result".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.query_in".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.response_out".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.query_quit".format(ROBOT_TYPE_NAME))

In [1]:
from __future__ import print_function
import os
RNB_PLANNING_DIR = os.environ["RNB_PLANNING_DIR"]
os.chdir(os.path.join(RNB_PLANNING_DIR, 'src'))
from pkg.controller.combined_robot import *
from pkg.utils.utils import get_now, try_mkdir
import subprocess

DATA_PATH = os.path.join(RNB_PLANNING_DIR, "data")
try_mkdir(DATA_PATH)

TEST_DATA_PATH = os.path.join(DATA_PATH, "stowing")
try_mkdir(TEST_DATA_PATH)

TEST_RESULT_PATH = os.path.join(DATA_PATH, "stowing-result")
try_mkdir(TEST_RESULT_PATH)

HEADS = ["plan_time", "length", "MP_count", "failed_MPs", "success"]

In [2]:
RTYPE = "indy7"
DAT_DIR = '20210613-035257'
FILE_OPTION = "obj_3_pole"
# DAT_DIR, FILE_OPTION = "20210613-023004", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-114040", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-151342", "obj_3"
# DAT_DIR, FILE_OPTION = "20210612-161536", "obj_3_pole"

# RTYPE = "panda"
# DAT_DIR = '20210613-035119'
# FILE_OPTION = "obj_1"
# # DAT_DIR, FILE_OPTION = "20210613-022832", "obj_1"
# # # DAT_DIR, FILE_OPTION = "20210612-221934", "obj_1"

CNAME_LIST = ['None','Tool','ToolReach','Full'] #,'Pairwise']
GRASP_SAMPLE = 10
VISUALIZE = False
PLAY_RESULT = False
TIMEOUT_MOTION = 5
MAX_TIME = 100
SHOW_STATE = False


### test

In [3]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

### load results

In [4]:
res_dict_all = defaultdict(lambda: defaultdict(list))
resfile_list = sorted(os.listdir(os.path.join(RESULTSET_PATH)))
for resfile in resfile_list:
    args = resfile[:-4].split("_")[1:]
    f_option = "_".join(args[:-2])
    fid, cname = args[-2:]
    resdat = load_pickle(os.path.join(RESULTSET_PATH, resfile))
    assert len(res_dict_all[f_option][cname]) == int(fid)
    res_dict_all[f_option][cname].append([resdat[head] for head in HEADS])

In [6]:
exp_list = ['obj_1', 'obj_3', 'obj_3_pole']
for ekey in exp_list:
    if ekey not in res_dict_all:
        continue
    res_dict = res_dict_all[ekey]
    print("="*30 + " {:<15} ".format(ekey) + "="*30)
    print("{:<15}: {:<15} {:<15} {:<15} {:<15} {:<15} {:<15}".format("Checker", "Planning time", "Motion count", "MP trials", "Failed MP", "Failures", "Total"))
    for cname in CNAME_LIST:
        if cname in res_dict:
            vv = np.array(res_dict[cname])[:10,:]
            if len(vv)>0:
                succ_vec = vv[:,-1]
                fail_vec = np.logical_not(succ_vec)
                idc_succ = np.where(succ_vec)[0]
                print(
                    "{:<15}: {:<15} {:<15} {:<15} {:<15} {:<15} {:<15}".format(cname,
                                    *(list(np.round(np.mean(vv[idc_succ, :-1], axis=0), 2)) \
                                      +[np.sum(fail_vec), len(vv)])))
    print(" ")

============================== obj_1           ==============================
Checker        : Planning time   Motion count    MP trials       Failed MP       Failures        Total          
None           : 7.2             5.8             3.4             0.0             0               10             
Tool           : 7.83            6.5             4.5             0.3             0               10             
ToolReach      : 3.55            5.8             4.2             0.3             0               10             
Full           : 1.94            5.4             3.2             0.0             0               10             
 
============================== obj_3           ==============================
Checker        : Planning time   Motion count    MP trials       Failed MP       Failures        Total          
None           : 23.27           8.11            4.56            0.0             1               10             
Tool           : 11.77           5.5             6.

## Res - panda
```

```

## Res - indy7
```
============================== obj_1           ==============================
Checker        : Planning time   Motion count    MP trials       Failed MP       Failures        Total          
None           : 7.2             5.8             3.4             0.0             0               10             
Tool           : 7.83            6.5             4.5             0.3             0               10             
ToolReach      : 3.55            5.8             4.2             0.3             0               10             
Full           : 1.94            5.4             3.2             0.0             0               10             
 
============================== obj_3           ==============================
Checker        : Planning time   Motion count    MP trials       Failed MP       Failures        Total          
None           : 23.27           8.11            4.56            0.0             1               10             
Tool           : 11.77           5.5             6.12            0.12            2               10             
ToolReach      : 5.02            5.57            9.0             0.0             3               10             
Full           : 6.66            5.44            3.56            0.0             1               10             
 
============================== obj_3_pole      ==============================
Checker        : Planning time   Motion count    MP trials       Failed MP       Failures        Total          
None           : 48.29           9.0             5.0             0.0             4               10             
Tool           : 15.81           6.0             11.75           1.12            2               10             
ToolReach      : 31.28           7.0             14.67           1.17            4               10             
Full           : 15.32           5.0             5.0             0.0             5               10            
```